In [1]:
import os
import glob
import json
import pickle
import numpy as np
import pandas as pd
import datasetdatabase as dsdb

print(dsdb.version.VERSION)

os.environ["DSDB_PROCESS_LIMIT"] = "16"

1.0.31.stable


Creating a simple df

In [2]:
raw_data = {
    'first_name': ['Jason', 'Molly', 'Tina', 'Jake', 'Amy'], 
    'last_name': ['Miller', 'Jacobson', 'Ali', 'Milner', 'Cooze'], 
    'age': [42, 52, 36, 24, 73], 
    'preTestScore': [4, 24, 31, 2, 3],
    'postTestScore': [25, 94, 57, 62, 70]}
df = pd.DataFrame(raw_data, columns = ['first_name', 'last_name', 'age', 'preTestScore', 'postTestScore'])
df

,first_name,last_name,age,preTestScore,postTestScore
0,Jason,Miller,42,4,25
1,Molly,Jacobson,52,24,94
2,Tina,Ali,36,31,57
3,Jake,Milner,24,2,62
4,Amy,Cooze,73,3,70


Upload to database

In [4]:
prod = dsdb.DatasetDatabase(config="/allen/aics/assay-dev/Analysis/QCB_database/prod_config.json")

Uploading dataset

In [5]:
ds = dsdb.Dataset(df, name="viana_apply_test", description="Testing the apply function to update a dataset")
ds.upload_to(prod)

Storing algorithm parameters.
Input dataset already exists in database. 1
Tearing down object...
[============================================================] 100.0% 5/5 ~ 0:0:0 remaining


Update the df

In [14]:
df["NewScore"] = 0
df

,first_name,last_name,age,preTestScore,postTestScore,NewScore
0,Jason,Miller,42,4,25,0
1,Molly,Jacobson,52,24,94,0
2,Tina,Ali,36,31,57,0
3,Jake,Milner,24,2,62,0
4,Amy,Cooze,73,3,70,0


### Apply function

```
# create a local database and upload initial dataset

local = dsdb.DatasetDatabase(dsdb.LOCAL, build=True)
volume = dsdb.Dataset(“volume.csv”)
volume.upload_to(local)

def normalize_column(dataset, name):
   # access the underlying data through the ds attribute
   col = dataset.ds[name]
   col = (col - col.mean()) / (col.max() - col.min())
   
   # reassign the column with the updated values
   dataset.ds[name] = col
   return dataset

volume.apply(normalize_column,
             algorithm_parameters={“name”: “volume”},
             output_dataset_name=”normalized volume”)
```

In [15]:
def update_ds(ds_old, df_new):
    ds_old.ds = df_new
    return ds_old

ds.apply(update_ds,
            algorithm_parameters={"df_new": df},
            output_dataset_name="viana_apply_test",
            algorithm_version="v1.0")

Storing algorithm parameters.
Input dataset already exists in database. 1
Input dataset already exists in database. 111
Storing algorithm parameters.
Input dataset already exists in database. 113
Dataset processing has begun...


AttributeError: can't set attribute

In [19]:
prod.get_dataset(id=113)

Reconstructing dataset...
[============================================================] 100.0% 1/1 ~ 0:0:0 remaining


info: {'id': 113, 'name': '907e4916-e019-4b71-b4c2-e9fd35832f2c', 'description': 'algorithm parameters', 'introspector': 'datasetdatabase.introspect.dictionary.DictionaryIntrospector', 'created': datetime.datetime(2018, 10, 31, 17, 1, 59, 87144)}
ds: True
name: 907e4916-e019-4b71-b4c2-e9fd35832f2c
description: algorithm parameters
introspector: <class 'datasetdatabase.introspect.dictionary.DictionaryIntrospector'>
md5: 3fd0b7e47e9c65e59835e5784f7ee276
sha256: d9e1cd1d781e8eeb7c8d5909bf61e832674911434df75d0835b8dec8fc1036b6
validated: True
annotations: []